In [ ]:
from collections import deque

class Graph:
    def __init__(self):
        self.adj_list = {}
        self.time = 0  # Global time counter for DFS

    def add_vertex(self, vertex):
        """Add a vertex to the graph if it doesn't already exist, initializing with default properties for DFS."""
        if vertex not in self.adj_list:
            self.adj_list[vertex] = {
                'neighbors': [],  # List of adjacent vertices
                'discovered': None,  # Time when this node was first discovered
                'finished': None,    # Time when the exploration of this node and its neighbors is complete
                'parent': None     # Parent node in the DFS tree
            }

    def add_edge(self, from_vertex, to_vertex, bidirectional=True):
        """
        Add an edge between two vertices. If bidirectional is True, adds an edge in both directions.

        Parameters:
            from_vertex (int): The starting vertex of the edge.
            to_vertex (int): The ending vertex of the edge.
            bidirectional (bool): If True, creates a bidirectional edge (default is True).

        This method also ensures that each vertex exists in the adjacency list and properly updates their neighbors.
        """
        if from_vertex == to_vertex:
            raise ValueError("Self-loops are not allowed")

        if from_vertex not in self.adj_list:
            self.add_vertex(from_vertex)
        if to_vertex not in self.adj_list:
            self.add_vertex(to_vertex)

        # Avoid adding the edge if it already exists
        if to_vertex not in self.adj_list[from_vertex]['neighbors']:
            self.adj_list[from_vertex]['neighbors'].append(to_vertex)

        if bidirectional:
            # Similarly, avoid adding the reverse edge if it already exists
            if from_vertex not in self.adj_list[to_vertex]['neighbors']:
                self.adj_list[to_vertex]['neighbors'].append(from_vertex)

    def bfs(self, start, use_deque=True):
        """
        Perform BFS using either a deque or a list based on the 'use_deque' parameter,
        marking the order of visited nodes and updating the depth and parent of each node.

        Parameters:
            start (int): The starting node for BFS.
            use_deque (bool): If True, uses a deque for efficient queue operations (recommended).
                              If False, uses a list which is less efficient due to pop(0) operations.

        Returns:
            tuple: A tuple containing a list of nodes representing the order in which they were visited
                  and a dictionary of node depths.
        """
        visited = set()   # Set to keep track of visited nodes
        queue = deque([start]) if use_deque else [start]  # Queue for BFS, deque or list based on use_deque
        result = []  # List to store the order of visited nodes
        depth = {start: 0}  # Dictionary to store the depth of each node from the start node
        enqued = set([start])
        self.adj_list[start]['parent'] = None  # Set the parent of the start node to None

        while queue:
            vertex = queue.popleft() if use_deque else queue.pop(0)  # Remove and return the leftmost element
            if vertex not in visited:
                visited.add(vertex)  # Mark the vertex as visited when it is dequeued
                result.append(vertex)  # Add vertex to the result list
                current_depth = depth[vertex]  # Get the current depth of the vertex

                for neighbor in self.adj_list[vertex]['neighbors']:
                    if neighbor not in visited and neighbor not in enqued:  # Check if neighbor is neither visited nor in queue
                        queue.append(neighbor)  # Add neighbor to the queue
                        enqued.add(neighbor)
                        self.adj_list[neighbor]['parent'] = vertex  # Update the parent of the neighbor
                        depth[neighbor] = current_depth + 1  # Set the depth of the neighbor

        return result, depth  # Return the order of visitation and the depths of nodes

    def bfs_find(self, start, target, use_deque=True, save_space=True):
        """
        Determine if a target node can be reached from the start node using BFS. Allows choice
        between using a deque for efficient queue operations or a list which may be less efficient.
        The `save_space` parameter controls whether to check for duplicates in the queue, which can
        save memory at the cost of increased computational overhead.

        Parameters:
            start (int): The starting node.
            target (int): The target node.
            use_deque (bool): If True, uses a deque for efficient queue operations (recommended).
                              If False, uses a list which is less efficient due to pop(0) operations.
            save_space (bool): If True, avoids duplicating nodes in the queue (default True).

        Returns:
            bool: True if the target is reachable from the start, False otherwise.
        """
        if start not in self.adj_list or target not in self.adj_list:
            return False  # Check if start and target nodes exist in the graph

        visited = set()  # Set to keep track of nodes that have been processed
        queue = deque([start]) if use_deque else [start]  # Initialize queue with the start node

        # Optional set to track enqueued nodes to avoid redundancy in the queue
        enqueued = set([start]) if save_space else None

        while queue:
            vertex = queue.popleft() if use_deque else queue.pop(0)  # Dequeue the first element
            if vertex == target:
                return True  # Return true if the target is found, indicating reachability
            visited.add(vertex)  # Mark the current vertex as visited after dequeue

            for neighbor in self.adj_list[vertex]['neighbors']:  # Explore each neighbor of the vertex
                if save_space:
                    # Check both visited and enqueued sets if save_space is enabled
                    if neighbor not in visited and neighbor not in enqueued:
                        queue.append(neighbor)  # Enqueue unvisited neighbor
                        enqueued.add(neighbor)  # Add to enqueued set to avoid future duplications
                else:
                    # Only check the visited set if save_space is disabled
                    if neighbor not in visited:
                        queue.append(neighbor)  # Enqueue unvisited neighbor without checking enqueued set

        return False  # Return false if the queue is exhausted without finding the target

    def dfs(self, start, recursion=True):
        """
        Perform a Depth First Search (DFS) starting from a given node.
        Handles both connected and disconnected graphs by ensuring all nodes are visited.
        Supports both recursive and iterative approaches.

        Parameters:
            start (int): The starting node for DFS.
            recursion (bool): If True, uses recursive DFS; if False, uses iterative DFS with a stack.

        Returns:
            list: A list of nodes in the order they were visited.
        """
        self.time = 0  # Reset time counter for each DFS run
        visited = []  # List to store the order of visited nodes for both approaches

        if recursion:
            def dfs_visit(vertex):
                """Recursive DFS function to visit nodes."""
                self.adj_list[vertex]['discovered'] = self.time
                self.time += 1
                visited.append(vertex)

                for neighbor in self.adj_list[vertex]['neighbors']:
                    if self.adj_list[neighbor]['discovered'] is None:
                        self.adj_list[neighbor]['parent'] = vertex
                        dfs_visit(neighbor)

                self.adj_list[vertex]['finished'] = self.time
                self.time += 1

            # Start DFS from the given start node if it hasn't been visited yet
            if self.adj_list[start]['discovered'] is None:
                dfs_visit(start)

            # Handle other unvisited nodes to cover disconnected graphs
            for vertex in self.adj_list:
                if self.adj_list[vertex]['discovered'] is None:
                    dfs_visit(vertex)

        else:
            visited_set = set()  # Set to track visited nodes for iterative DFS

            # Handle disconnected graphs by iterating over all vertices
            for vertex in self.adj_list:
                if vertex not in visited_set:
                    stack = [(vertex, 'enter')]
                    while stack:
                        current_vertex, state = stack.pop()
                        if state == 'enter':
                            if current_vertex not in visited_set:
                                visited.append(current_vertex)
                                visited_set.add(current_vertex)
                                self.adj_list[current_vertex]['discovered'] = self.time
                                self.time += 1
                                stack.append((current_vertex, 'exit'))
                                for neighbor in reversed(self.adj_list[current_vertex]['neighbors']):
                                    if neighbor not in visited_set:
                                        stack.append((neighbor, 'enter'))
                                        # Check if parent is not set before setting it
                                        if self.adj_list[neighbor]['parent'] is None:
                                            self.adj_list[neighbor]['parent'] = current_vertex
                        elif state == 'exit':
                            self.adj_list[current_vertex]['finished'] = self.time
                            self.time += 1
                        else:# Handle unexpected state
                            raise ValueError(f"Unexpected state encountered: {state}")

        return visited  # Return the list of all visited nodes

    def reconstruct_path(self, target):
        """
        Reconstruct the path from the start node to the target node using parent pointers set during graph traversal.

        Parameters:
            target (int or str): The target node to trace the path back to the start node.

        Returns:
            list: A list representing the path from the start node to the target node, in the correct order from start to end.
        """
        if target not in self.adj_list:
            raise ValueError(f"Target {target} not found in the graph.")
        if self.adj_list[target]['parent'] is None:
            raise ValueError("Target node has not been reached by the traversal OR target node = start node.")

        path = []
        step = target
        # Traverse backwards from the target using the parent links stored in the adjacency list
        while step is not None:
            path.append(step)
            step = self.adj_list[step]['parent']  # Access the parent directly from the adjacency list
        path.reverse()  # Reverse to start from the initial node
        return path

    def classify_edges(self, count_bidir_once=True):
        """
        Classify edges in the graph based on their roles as revealed during a Depth-First Search (DFS) traversal.

        This method categorizes each edge into one of four types, utilizing the discovery and completion times,
        as well as parent-child relationships established by a prior DFS traversal:
        - Tree Edges: Edges that form the spanning tree of DFS, directly connecting a parent and its child.
        - Back Edges: Edges that connect a vertex to an ancestor, not in the direct path, indicating cycles.
        - Forward Edges: Edges that connect a vertex to a descendant beyond the direct children.
        - Cross Edges: Edges that connect vertices across different DFS trees or subtrees, not involving direct ancestor-descendant relationships.

        The classification depends on properties set during DFS (discovered, finished, parent), hence a DFS must be run prior to this method.

        Note: The classification of edges is contingent upon the path taken by the DFS traversal, which is influenced by the starting node of the DFS. Different starting points may lead to different edge classifications.

        Parameters:
            count_bidir_once (bool): If True, bidirectional edges are counted once, suitable for undirected graphs. If False, edges are counted per direction, suitable for directed graphs.

        Returns:
            dict: A dictionary containing lists of edges classified by type ('tree', 'back', 'forward', 'cross').
        """
        edge_types = {
            'tree': [],
            'back': [],
            'forward': [],
            'cross': []
        }
        visited_edges = set()

        for start_node in self.adj_list:
            for end_node in self.adj_list[start_node]['neighbors']:
                edge = (start_node, end_node)
                reversed_edge = (end_node, start_node)

                if count_bidir_once and reversed_edge in visited_edges:
                    continue  # Skip this edge if it has already been processed from the other direction

                if self.adj_list[end_node]['parent'] == start_node:
                    edge_types['tree'].append((start_node, end_node))
                elif self.adj_list[end_node]['discovered'] is not None and self.adj_list[end_node]['discovered'] < self.adj_list[start_node]['discovered'] and self.adj_list[end_node]['finished'] > self.adj_list[start_node]['discovered']:
                    edge_types['back'].append((start_node, end_node))
                elif self.adj_list[end_node]['discovered'] > self.adj_list[start_node]['discovered'] and self.adj_list[end_node]['finished'] < self.adj_list[start_node]['finished']:
                    edge_types['forward'].append((start_node, end_node))
                elif self.adj_list[end_node]['finished'] < self.adj_list[start_node]['discovered'] or self.adj_list[start_node]['finished'] < self.adj_list[end_node]['discovered']:
                    edge_types['cross'].append((start_node, end_node))

                if count_bidir_once:
                    visited_edges.add(edge)
                    visited_edges.add(reversed_edge)  # Ensure to mark both directions as processed

        return edge_types

    def topological_sort(self):
        """
        Perform topological sorting on the graph to produce a linear ordering of vertices
        such that for every directed edge (u, v), vertex 'u' appears before vertex 'v' in the order.
        This method uses Depth-First Search (DFS) to ensure all nodes are visited and their finish times
        are recorded for sorting.

        The function begins by ensuring all nodes have been visited through DFS. It then checks for cycles in the graph.
        If any back edges are found, it indicates a cycle, which means topological sorting is not possible.
        Otherwise, it sorts vertices based on their DFS finish times in descending order to produce a valid topological order.

        Returns:
            tuple:
                - If there's a cycle, returns `None` with a message indicating that topological sort isn't possible.
                - If there's no cycle, returns a list of sorted nodes based on finish times and a success message.
        """
        # Complete DFS traversal
        # Ensures all nodes are visited, even if they are in disconnected parts
        for vertex in self.adj_list:
            if self.adj_list[vertex]['discovered'] is None:
                self.dfs(vertex)

        # Check if the graph contains cycles
        edge_types = self.classify_edges()
        if len(edge_types['back']) > 0:
            return None, "Cycle detected, topological sort not possible"

        # Sort vertices based on finish times for topological sorting
        sorted_nodes = sorted(self.adj_list, key=lambda x: self.adj_list[x]['finished'], reverse=True)
        return sorted_nodes, "Topological sort successful"

    def reverse_graph(self):
        """
        Create and return a new Graph instance that represents the reverse of the current graph.
        In the reversed graph, all the directions of the edges are inverted such that for every
        original directed edge (u, v), the reversed graph will have an edge (v, u).

        This function iterates through each vertex and its adjacency list in the current graph's
        adjacency list, adding a reverse directed edge for each found edge to the new graph instance.
        This effectively reverses the direction of graph traversal. The newly created vertices in the
        reversed graph have their 'discovered', 'finished', and 'parent' attributes initialized to None,
        as they are newly instantiated and not yet part of any graph traversal process.

        This is particularly useful for algorithms that need to explore the graph in the reverse
        direction, such as algorithms for finding strongly connected components.

        Returns:
            Graph: A new Graph instance representing the reversed graph, with all vertices and
                  reversed edges from the original graph. Each vertex in this new graph starts
                  with traversal-related attributes ('discovered', 'finished', 'parent') set to None.

        Example:
            If the original graph has edges A -> B and A -> C, the reversed graph will have edges
            B -> A and C -> A.

        Usage:
            reversed_graph = original_graph.reverse_graph()
        """
        reversed_graph = Graph()  # New instance of Graph for the reversed graph
        for vertex in self.adj_list:
            for neighbor in self.adj_list[vertex]['neighbors']:
                reversed_graph.add_edge(neighbor, vertex)  # Add the reverse of each edge
        return reversed_graph

    def find_sccs(self):
        """
        Finds all maximally strongly connected components (MSCCs) of the graph using Kosaraju's algorithm.
        This method involves several key steps:

        1. Perform an initial depth-first search (DFS) to determine the finish times of each vertex.
        2. Sort all vertices in descending order based on their finish times to prepare for the second DFS pass.
        3. Reverse the graph, which involves reversing the direction of all edges.
        4. Perform a second DFS based on the order determined in step 2. Each DFS run in this step identifies a single SCC.

        The algorithm takes advantage of the properties of graph reversal and the order of traversal to efficiently
        identify SCCs. It guarantees that each SCC is visited exactly once in the reversed graph traversal, thereby
        collecting all vertices that are mutually reachable.

        Returns:
            list of lists: Each inner list contains the vertices of one strongly connected component.
        """
        # Step 1: Perform DFS to get finish times
        self.dfs(start=list(self.adj_list.keys())[0])  # Starting from an arbitrary node

        # Step 2: Sort vertices by descending finish times
        sorted_vertices = sorted(self.adj_list, key=lambda v: self.adj_list[v]['finished'], reverse=True)

        # Step 3: Reverse the graph
        reversed_graph = self.reverse_graph()

        # Step 4: Perform DFS on the reversed graph using the sorted order of vertices
        visited = set()
        sccs = []

        def dfs_kosaraju(vertex, collected):
            """ Helper function to perform DFS in Kosaraju's algorithm and collect vertices of one SCC """
            visited.add(vertex)
            collected.append(vertex)
            for neighbor in reversed_graph.adj_list[vertex]['neighbors']:
                if neighbor not in visited:
                    dfs_kosaraju(neighbor, collected)

        for vertex in sorted_vertices:
            if vertex not in visited:
                current_scc = []
                dfs_kosaraju(vertex, current_scc)
                sccs.append(current_scc)

        return sccs

    def __str__(self):
        lines = ["Graph adjacency list with vertex properties:"]
        for vertex, properties in self.adj_list.items():
            line = f"{vertex}: Neighbors -> {properties['neighbors']}, "
            line += f"Discovered -> {properties['discovered']}, "
            line += f"Finished -> {properties['finished']}, "
            line += f"Parent -> {properties['parent']}"
            lines.append(line)
        return '\n'.join(lines)

In [ ]:
import heapq
from collections import deque

class WeightedGraph:
    def __init__(self, default_weight=1):
        """
        Initialize a new weighted graph.

        Parameters:
            default_weight (int or float): The default weight to use for edges where no specific weight is provided.
        """
        self.adj_list = {}
        self.time = 0  # Global time counter for DFS
        self.default_weight = default_weight

    def add_vertex(self, vertex):
        """
        Add a vertex to the weighted graph if it doesn't already exist, initializing with default properties for DFS.

        Parameters:
            vertex (int or str): The vertex to add to the graph.
        """
        if vertex not in self.adj_list:
            self.adj_list[vertex] = {
                'neighbors': [],  # List of adjacent vertices with weights
                'discovered': None,  # Time when this node was first discovered
                'finished': None,    # Time when the exploration of this node and its neighbors is complete
                'parent': None     # Parent node in the DFS tree
            }

    def add_edge(self, from_vertex, to_vertex, weight=None, bidirectional=True):
        """
        Add a weighted edge between two vertices. If bidirectional is True, adds an edge in both directions.

        Parameters:
            from_vertex (int or str): The starting vertex of the edge.
            to_vertex (int or str): The ending vertex of the edge.
            weight (int or float, optional): The weight of the edge. If None, uses the default weight of the graph.
            bidirectional (bool): If True, creates a bidirectional edge (default is True).

        This method also ensures that each vertex exists in the adjacency list and properly updates their neighbors
        with the specified or default weight.
        """
        if from_vertex not in self.adj_list:
            self.add_vertex(from_vertex)
        if to_vertex not in self.adj_list:
            self.add_vertex(to_vertex)

        edge_weight = weight if weight is not None else self.default_weight

        # Update neighbor list with weight
        neighbor_info = (to_vertex, edge_weight)
        if neighbor_info not in self.adj_list[from_vertex]['neighbors']:
            self.adj_list[from_vertex]['neighbors'].append(neighbor_info)

        if bidirectional:
            reverse_neighbor_info = (from_vertex, edge_weight)
            if reverse_neighbor_info not in self.adj_list[to_vertex]['neighbors']:
                self.adj_list[to_vertex]['neighbors'].append(reverse_neighbor_info)

    @staticmethod
    def convert_to_weighted(unweighted_graph, default_weight=1):
        """
        Converts an unweighted graph (instance of Graph) to a weighted graph, applying a default weight to all edges.

        Parameters:
            unweighted_graph (Graph): An instance of the unweighted Graph class to be converted.
            default_weight (int or float): The weight to apply to every edge in the new weighted graph.

        Returns:
            WeightedGraph: A new instance of WeightedGraph containing all vertices and edges from the unweighted graph,
                           with each edge assigned the specified default weight.
        """
        weighted_graph = WeightedGraph(default_weight)
        for vertex, properties in unweighted_graph.adj_list.items():
            weighted_graph.add_vertex(vertex)
            for neighbor in properties['neighbors']:
                # Add each edge with the default weight, assume unidirectional to match original graph's structure
                weighted_graph.add_edge(vertex, neighbor, default_weight, bidirectional=False)
        return weighted_graph

    def __str__(self):
        """
        Provide a string representation of the graph's adjacency list, including vertex properties.

        Returns:
            str: A string that lists each vertex and its neighbors with weights, and includes DFS properties like discovered time, finished time, and parent.
        """
        lines = ["Weighted Graph adjacency list with vertex properties:"]
        for vertex, properties in self.adj_list.items():
            # Create a string that lists the neighbors and weights
            neighbors_str = ", ".join([f"{neighbor} (weight {weight})" for neighbor, weight in properties['neighbors']])

            # Format the string with other DFS properties
            line = f"{vertex}: Neighbors -> [{neighbors_str}], "
            line += f"Discovered -> {properties['discovered']}, "
            line += f"Finished -> {properties['finished']}, "
            line += f"Parent -> {properties['parent']}"
            lines.append(line)
        return '\n'.join(lines)

    def kruskals_mst(self):
        """
        Compute the Minimum Spanning Tree (MST) of the graph using Kruskal's algorithm.

        Kruskal's algorithm processes edges in ascending order by weight, ensuring that the smallest edges
        are considered first, without forming cycles, to construct the MST with the minimum possible total weight.
        This implementation utilizes the Union-Find data structure with path compression and union by rank to manage subsets of vertices efficiently.

        Returns:
            list of tuples: A list of edges in the MST in the form (weight, from_vertex, to_vertex).

        Details:
        - **Initialization**: Start with each vertex as its own subset.
        - **Edge Processing**: Iterate over edges sorted by weight and merge subsets if they are not already connected, ensuring no cycles.
        - **Union-Find Optimization**: Path compression and union by rank speed up the subset operations, keeping the tree structures flat.
        - **Early Termination**: The algorithm stops once the MST contains exactly V-1 edges, where V is the number of vertices in the graph.
        """
        # Create a list of all edges in the graph, ensuring each edge is listed once
        edges = []
        for vertex, props in self.adj_list.items():
            for neighbor, weight in props['neighbors']:
                if vertex < neighbor:  # Ensure each edge is added only once in an undirected graph
                    edges.append((weight, vertex, neighbor))

        # Sort edges by weight to process the smallest edges first
        edges.sort()

        # Initialize union-find structure to manage subsets of vertices
        parent = {}
        rank = {}

        def find(v):
            # Find the root of the subset containing 'v', with path compression
            if parent[v] != v:
                parent[v] = find(parent[v])
            return parent[v]

        def union(v1, v2):
            # Union two subsets containing 'v1' and 'v2' by rank
            root1 = find(v1)
            root2 = find(v2)
            if root1 != root2:
                if rank[root1] > rank[root2]:
                    parent[root2] = root1  # Make root1 the parent of root2
                else:
                    parent[root1] = root2  # Make root2 the parent of root1
                    if rank[root1] == rank[root2]:
                        rank[root2] += 1  # Increment the rank if both roots had the same rank

        # Initialize each vertex to be its own subset with rank 0
        for vertex in self.adj_list:
            parent[vertex] = vertex
            rank[vertex] = 0

        mst = []  # List to hold the edges of the MST
        for weight, u, v in edges:
            # For each edge, if u and v are in different subsets, include the edge in the MST
            if find(u) != find(v):
                union(u, v)
                mst.append((weight, u, v))
                # Stop if the MST has the right amount of edges
                if len(mst) == len(self.adj_list) - 1:
                    break

        return mst  # Return the list of MST edges

    def bellman_ford(self, source):
        """
        Computes the shortest path from a single source to all other vertices using the Bellman-Ford algorithm.
        This method can handle negative weights and will also detect negative weight cycles.

        Parameters:
            source (int or str): The starting vertex for path calculations.

        Returns:
            tuple: A tuple containing two elements:
                1. A dictionary where keys are the vertices and values are the shortest distance from the source.
                2. A boolean indicating whether a negative weight cycle has been detected.
        """
        # Step 1: Initialize distances from source to all vertices as infinite except the source itself
        distances = {vertex: float('inf') for vertex in self.adj_list}
        distances[source] = 0
        parents = {vertex: None for vertex in self.adj_list}

        # Step 2: Relax all edges |V| - 1 times. A simple shortest path from source to any other vertex can have at most |V| - 1 edges
        for _ in range(len(self.adj_list) - 1):
            for vertex in self.adj_list:
                for neighbor, weight in self.adj_list[vertex]['neighbors']:
                    if distances[vertex] + weight < distances[neighbor]:
                        distances[neighbor] = distances[vertex] + weight
                        parents[neighbor] = vertex

        # Step 3: Check for negative weight cycles. The above step guarantees shortest paths if graph doesn't contain negative weight cycle
        # If we get a shorter path, then there is a cycle
        cycle_detected = False
        for vertex in self.adj_list:
            for neighbor, weight in self.adj_list[vertex]['neighbors']:
                if distances[vertex] + weight < distances[neighbor]:
                    cycle_detected = True
                    break
            if cycle_detected:
                break

        return distances, cycle_detected

    def dijkstra(self, source):
        """
        Compute the shortest paths from the source to all other vertices using Dijkstra's algorithm.

        Parameters:
            source (int or str): The starting vertex.

        Returns:
            tuple: A tuple containing two dictionaries:
                1. Distances: keys are the vertices and values are the shortest distances from the source.
                2. Parents: keys are the vertices and values are the predecessors in the path from the source.
        """
        # Initialize distances and parents
        distances = {vertex: float('inf') for vertex in self.adj_list}
        distances[source] = 0
        parents = {vertex: None for vertex in self.adj_list}

        # Priority queue to hold vertices to be processed, initialized with the source
        priority_queue = [(0, source)]
        visited = set()

        while priority_queue:
            # Get the vertex with the smallest distance
            current_distance, current_vertex = heapq.heappop(priority_queue)

            if current_vertex in visited:
                continue
            visited.add(current_vertex)

            # Process each adjacent vertex
            for neighbor, weight in self.adj_list[current_vertex]['neighbors']:
                if neighbor in visited:
                    continue
                new_distance = current_distance + weight
                # Only consider this new path if it's better
                if new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    parents[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (new_distance, neighbor))

        return distances, parents

    def bfs(self, start, use_deque=True):
        """
        Perform BFS using either a deque or a list based on the 'use_deque' parameter,
        marking the order of visited nodes and updating the depth and parent of each node.

        Parameters:
            start (int): The starting node for BFS.
            use_deque (bool): If True, uses a deque for efficient queue operations (recommended).
                              If False, uses a list which is less efficient due to pop(0) operations.

        Returns:
            tuple: A tuple containing a list of nodes representing the order in which they were visited
                  and a dictionary of node depths.
        """
        visited = set()   # Set to keep track of visited nodes
        queue = deque([start]) if use_deque else [start]  # Queue for BFS, deque or list based on use_deque
        result = []  # List to store the order of visited nodes
        depth = {start: 0}  # Dictionary to store the depth of each node from the start node
        enqued = set([start])
        self.adj_list[start]['parent'] = None  # Set the parent of the start node to None

        while queue:
            vertex = queue.popleft() if use_deque else queue.pop(0)  # Remove and return the leftmost element
            if vertex not in visited:
                visited.add(vertex)  # Mark the vertex as visited when it is dequeued
                result.append(vertex)  # Add vertex to the result list
                current_depth = depth[vertex]  # Get the current depth of the vertex

                for neighbor in self.adj_list[vertex]['neighbors']:
                    if neighbor not in visited and neighbor not in enqued:  # Check if neighbor is neither visited nor in queue
                        queue.append(neighbor)  # Add neighbor to the queue
                        enqued.add(neighbor)
                        self.adj_list[neighbor]['parent'] = vertex  # Update the parent of the neighbor
                        depth[neighbor] = current_depth + 1  # Set the depth of the neighbor

        return result, depth  # Return the order of visitation and the depths of nodes

    def bfs_find(self, start, target, use_deque=True, save_space=True):
        """
        Determine if a target node can be reached from the start node using BFS. Allows choice
        between using a deque for efficient queue operations or a list which may be less efficient.
        The `save_space` parameter controls whether to check for duplicates in the queue, which can
        save memory at the cost of increased computational overhead.

        Parameters:
            start (int): The starting node.
            target (int): The target node.
            use_deque (bool): If True, uses a deque for efficient queue operations (recommended).
                              If False, uses a list which is less efficient due to pop(0) operations.
            save_space (bool): If True, avoids duplicating nodes in the queue (default True).

        Returns:
            bool: True if the target is reachable from the start, False otherwise.
        """
        if start not in self.adj_list or target not in self.adj_list:
            return False  # Check if start and target nodes exist in the graph

        visited = set()  # Set to keep track of nodes that have been processed
        queue = deque([start]) if use_deque else [start]  # Initialize queue with the start node

        # Optional set to track enqueued nodes to avoid redundancy in the queue
        enqueued = set([start]) if save_space else None

        while queue:
            vertex = queue.popleft() if use_deque else queue.pop(0)  # Dequeue the first element
            if vertex == target:
                return True  # Return true if the target is found, indicating reachability
            visited.add(vertex)  # Mark the current vertex as visited after dequeue

            for neighbor in self.adj_list[vertex]['neighbors']:  # Explore each neighbor of the vertex
                if save_space:
                    # Check both visited and enqueued sets if save_space is enabled
                    if neighbor not in visited and neighbor not in enqueued:
                        queue.append(neighbor)  # Enqueue unvisited neighbor
                        enqueued.add(neighbor)  # Add to enqueued set to avoid future duplications
                else:
                    # Only check the visited set if save_space is disabled
                    if neighbor not in visited:
                        queue.append(neighbor)  # Enqueue unvisited neighbor without checking enqueued set

        return False  # Return false if the queue is exhausted without finding the target

    def dfs(self, start, recursion=True):
        """
        Perform a Depth First Search (DFS) starting from a given node.
        Handles both connected and disconnected graphs by ensuring all nodes are visited.
        Supports both recursive and iterative approaches.

        Parameters:
            start (int): The starting node for DFS.
            recursion (bool): If True, uses recursive DFS; if False, uses iterative DFS with a stack.

        Returns:
            list: A list of nodes in the order they were visited.
        """
        self.time = 0  # Reset time counter for each DFS run
        visited = []  # List to store the order of visited nodes for both approaches

        if recursion:
            def dfs_visit(vertex):
                """Recursive DFS function to visit nodes."""
                self.adj_list[vertex]['discovered'] = self.time
                self.time += 1
                visited.append(vertex)

                for neighbor in self.adj_list[vertex]['neighbors']:
                    if self.adj_list[neighbor]['discovered'] is None:
                        self.adj_list[neighbor]['parent'] = vertex
                        dfs_visit(neighbor)

                self.adj_list[vertex]['finished'] = self.time
                self.time += 1

            # Start DFS from the given start node if it hasn't been visited yet
            if self.adj_list[start]['discovered'] is None:
                dfs_visit(start)

            # Handle other unvisited nodes to cover disconnected graphs
            for vertex in self.adj_list:
                if self.adj_list[vertex]['discovered'] is None:
                    dfs_visit(vertex)

        else:
            visited_set = set()  # Set to track visited nodes for iterative DFS

            # Handle disconnected graphs by iterating over all vertices
            for vertex in self.adj_list:
                if vertex not in visited_set:
                    stack = [(vertex, 'enter')]
                    while stack:
                        current_vertex, state = stack.pop()
                        if state == 'enter':
                            if current_vertex not in visited_set:
                                visited.append(current_vertex)
                                visited_set.add(current_vertex)
                                self.adj_list[current_vertex]['discovered'] = self.time
                                self.time += 1
                                stack.append((current_vertex, 'exit'))
                                for neighbor in reversed(self.adj_list[current_vertex]['neighbors']):
                                    if neighbor not in visited_set:
                                        stack.append((neighbor, 'enter'))
                                        # Check if parent is not set before setting it
                                        if self.adj_list[neighbor]['parent'] is None:
                                            self.adj_list[neighbor]['parent'] = current_vertex
                        elif state == 'exit':
                            self.adj_list[current_vertex]['finished'] = self.time
                            self.time += 1
                        else:# Handle unexpected state
                            raise ValueError(f"Unexpected state encountered: {state}")

        return visited  # Return the list of all visited nodes

    def reconstruct_path(self, target):
        """
        Reconstruct the path from the start node to the target node using parent pointers set during graph traversal.

        Parameters:
            target (int or str): The target node to trace the path back to the start node.

        Returns:
            list: A list representing the path from the start node to the target node, in the correct order from start to end.
        """
        if target not in self.adj_list:
            raise ValueError(f"Target {target} not found in the graph.")
        if self.adj_list[target]['parent'] is None:
            raise ValueError("Target node has not been reached by the traversal OR target node = start node.")

        path = []
        step = target
        # Traverse backwards from the target using the parent links stored in the adjacency list
        while step is not None:
            path.append(step)
            step = self.adj_list[step]['parent']  # Access the parent directly from the adjacency list
        path.reverse()  # Reverse to start from the initial node
        return path

    def classify_edges(self, count_bidir_once=True):
        """
        Classify edges in the graph based on their roles as revealed during a Depth-First Search (DFS) traversal.

        This method categorizes each edge into one of four types, utilizing the discovery and completion times,
        as well as parent-child relationships established by a prior DFS traversal:
        - Tree Edges: Edges that form the spanning tree of DFS, directly connecting a parent and its child.
        - Back Edges: Edges that connect a vertex to an ancestor, not in the direct path, indicating cycles.
        - Forward Edges: Edges that connect a vertex to a descendant beyond the direct children.
        - Cross Edges: Edges that connect vertices across different DFS trees or subtrees, not involving direct ancestor-descendant relationships.

        The classification depends on properties set during DFS (discovered, finished, parent), hence a DFS must be run prior to this method.

        Note: The classification of edges is contingent upon the path taken by the DFS traversal, which is influenced by the starting node of the DFS. Different starting points may lead to different edge classifications.

        Parameters:
            count_bidir_once (bool): If True, bidirectional edges are counted once, suitable for undirected graphs. If False, edges are counted per direction, suitable for directed graphs.

        Returns:
            dict: A dictionary containing lists of edges classified by type ('tree', 'back', 'forward', 'cross').
        """
        edge_types = {
            'tree': [],
            'back': [],
            'forward': [],
            'cross': []
        }
        visited_edges = set()

        for start_node in self.adj_list:
            for end_node in self.adj_list[start_node]['neighbors']:
                edge = (start_node, end_node)
                reversed_edge = (end_node, start_node)

                if count_bidir_once and reversed_edge in visited_edges:
                    continue  # Skip this edge if it has already been processed from the other direction

                if self.adj_list[end_node]['parent'] == start_node:
                    edge_types['tree'].append((start_node, end_node))
                elif self.adj_list[end_node]['discovered'] is not None and self.adj_list[end_node]['discovered'] < self.adj_list[start_node]['discovered'] and self.adj_list[end_node]['finished'] > self.adj_list[start_node]['discovered']:
                    edge_types['back'].append((start_node, end_node))
                elif self.adj_list[end_node]['discovered'] > self.adj_list[start_node]['discovered'] and self.adj_list[end_node]['finished'] < self.adj_list[start_node]['finished']:
                    edge_types['forward'].append((start_node, end_node))
                elif self.adj_list[end_node]['finished'] < self.adj_list[start_node]['discovered'] or self.adj_list[start_node]['finished'] < self.adj_list[end_node]['discovered']:
                    edge_types['cross'].append((start_node, end_node))

                if count_bidir_once:
                    visited_edges.add(edge)
                    visited_edges.add(reversed_edge)  # Ensure to mark both directions as processed

        return edge_types

    def topological_sort(self):
        """
        Perform topological sorting on the graph to produce a linear ordering of vertices
        such that for every directed edge (u, v), vertex 'u' appears before vertex 'v' in the order.
        This method uses Depth-First Search (DFS) to ensure all nodes are visited and their finish times
        are recorded for sorting.

        The function begins by ensuring all nodes have been visited through DFS. It then checks for cycles in the graph.
        If any back edges are found, it indicates a cycle, which means topological sorting is not possible.
        Otherwise, it sorts vertices based on their DFS finish times in descending order to produce a valid topological order.

        Returns:
            tuple:
                - If there's a cycle, returns `None` with a message indicating that topological sort isn't possible.
                - If there's no cycle, returns a list of sorted nodes based on finish times and a success message.
        """
        # Complete DFS traversal
        # Ensures all nodes are visited, even if they are in disconnected parts
        for vertex in self.adj_list:
            if self.adj_list[vertex]['discovered'] is None:
                self.dfs(vertex)

        # Check if the graph contains cycles
        edge_types = self.classify_edges()
        if len(edge_types['back']) > 0:
            return None, "Cycle detected, topological sort not possible"

        # Sort vertices based on finish times for topological sorting
        sorted_nodes = sorted(self.adj_list, key=lambda x: self.adj_list[x]['finished'], reverse=True)
        return sorted_nodes, "Topological sort successful"

    def reverse_graph(self):
        """
        Create and return a new Graph instance that represents the reverse of the current graph.
        In the reversed graph, all the directions of the edges are inverted such that for every
        original directed edge (u, v), the reversed graph will have an edge (v, u).

        This function iterates through each vertex and its adjacency list in the current graph's
        adjacency list, adding a reverse directed edge for each found edge to the new graph instance.
        This effectively reverses the direction of graph traversal. The newly created vertices in the
        reversed graph have their 'discovered', 'finished', and 'parent' attributes initialized to None,
        as they are newly instantiated and not yet part of any graph traversal process.

        This is particularly useful for algorithms that need to explore the graph in the reverse
        direction, such as algorithms for finding strongly connected components.

        Returns:
            Graph: A new Graph instance representing the reversed graph, with all vertices and
                  reversed edges from the original graph. Each vertex in this new graph starts
                  with traversal-related attributes ('discovered', 'finished', 'parent') set to None.

        Example:
            If the original graph has edges A -> B and A -> C, the reversed graph will have edges
            B -> A and C -> A.

        Usage:
            reversed_graph = original_graph.reverse_graph()
        """
        reversed_graph = WeightedGraph()  # New instance of WeightedGraph for the reversed graph
        for vertex in self.adj_list:
            for neighbor, weight in self.adj_list[vertex]['neighbors']:
                reversed_graph.add_edge(neighbor, vertex, weight)  # Add the reverse of each edge with the original weight
        return reversed_graph

    def find_sccs(self):
        """
        Finds all maximally strongly connected components (MSCCs) of the graph using Kosaraju's algorithm.
        This method involves several key steps:

        1. Perform an initial depth-first search (DFS) to determine the finish times of each vertex.
        2. Sort all vertices in descending order based on their finish times to prepare for the second DFS pass.
        3. Reverse the graph, which involves reversing the direction of all edges.
        4. Perform a second DFS based on the order determined in step 2. Each DFS run in this step identifies a single SCC.

        The algorithm takes advantage of the properties of graph reversal and the order of traversal to efficiently
        identify SCCs. It guarantees that each SCC is visited exactly once in the reversed graph traversal, thereby
        collecting all vertices that are mutually reachable.

        Returns:
            list of lists: Each inner list contains the vertices of one strongly connected component.
        """
        # Step 1: Perform DFS to get finish times
        self.dfs(start=list(self.adj_list.keys())[0])  # Starting from an arbitrary node

        # Step 2: Sort vertices by descending finish times
        sorted_vertices = sorted(self.adj_list, key=lambda v: self.adj_list[v]['finished'], reverse=True)

        # Step 3: Reverse the graph
        reversed_graph = self.reverse_graph()

        # Step 4: Perform DFS on the reversed graph using the sorted order of vertices
        visited = set()
        sccs = []

        def dfs_kosaraju(vertex, collected):
            """ Helper function to perform DFS in Kosaraju's algorithm and collect vertices of one SCC """
            visited.add(vertex)
            collected.append(vertex)
            for neighbor in reversed_graph.adj_list[vertex]['neighbors']:
                if neighbor not in visited:
                    dfs_kosaraju(neighbor, collected)

        for vertex in sorted_vertices:
            if vertex not in visited:
                current_scc = []
                dfs_kosaraju(vertex, current_scc)
                sccs.append(current_scc)
        return sccs